In [58]:
import pickle
import re
import pandas as pd
import os

Try and gather the s-box info from sage. If it doesn't already exist, generate it.

In [59]:
try: 
    with open("../chipwhisperer_minimal/generate_c/sboxes_info.pkl", "rb") as f:
        sboxes_df = pickle.load(f)
except FileNotFoundError:
    print("No pickled sbox info found, generating by running sboxes_info.py")
    import sboxes_info
    sboxes_info.main()
    with open("../chipwhisperer_minimal/generate_c/sboxes_info.pkl", "rb") as f:
        sboxes_df = pickle.load(f)

Get all results files (.txt files)

In [60]:
files = []
for filename in os.scandir("./"):
    if filename.is_file():
        # Only grab the txt files
        if re.search(r'\.txt$', filename.name):
            files.append(filename.name)
files

['avg_leaks_tvla_cwlitearm.txt',
 'n_traces_dpa_cwnano.txt',
 'avg_leaks_tvla_cwnano_CBC.txt',
 'avg_leaks_tvla_cwlitearm_CTR.txt',
 '30run_avg_leaks_tvla_cwlitearm_ECB_1000.txt',
 'avg_leaks_tvla_cwnano.txt',
 '30run_avg_leaks_tvla_cwlitearm_ECB.txt',
 'n_traces_cpa_cwnano.txt',
 'n_traces_dpa_cwlitearm.txt',
 '30run_avg_leaks_tvla_cwnano_ECB_1000.txt',
 '30run_n_traces_cpa_cwlitearm_ECB.txt',
 'n_traces_cpa_cwlitearm.txt',
 'avg_leaks_tvla_cwlitearm_CBC.txt',
 'avg_leaks_tvla_cwnano_CTR.txt']

Get associated column names for each result

In [61]:
col_names = ["" for _ in range(len(files))]

regex = r'(\d+)?(run)?_?(n_traces|avg_leaks)_(tvla|dpa|cpa)_(cwnano|cwlitearm)_?(ECB|CBC|CTR)?_?(\d+)?'
for i, filename in enumerate(files):
    # Get captured patterns
    matches = re.findall(regex, filename)[0]

    # Replace empty strings with None
    matches = [None if x == '' else x for x in matches]
    print(matches)

    # Make associated column names
    num_run = matches[0] if matches[0] else "10"
    metric = matches[2]
    test = matches[3]
    device = matches[4]
    mode = matches[5] if matches[5] else "ECB"
    tvla_samples = matches[6] if matches[6] else "250"

    if test == "tvla":
        col_name = f"{num_run}_{device}_{test}_{mode}_{tvla_samples}"
    else:
        col_name = f"{num_run}_{device}_{test}_{mode}"

    col_names[i] = col_name

[None, None, 'avg_leaks', 'tvla', 'cwlitearm', None, None]
[None, None, 'n_traces', 'dpa', 'cwnano', None, None]
[None, None, 'avg_leaks', 'tvla', 'cwnano', 'CBC', None]
[None, None, 'avg_leaks', 'tvla', 'cwlitearm', 'CTR', None]
['30', 'run', 'avg_leaks', 'tvla', 'cwlitearm', 'ECB', '1000']
[None, None, 'avg_leaks', 'tvla', 'cwnano', None, None]
['30', 'run', 'avg_leaks', 'tvla', 'cwlitearm', 'ECB', None]
[None, None, 'n_traces', 'cpa', 'cwnano', None, None]
[None, None, 'n_traces', 'dpa', 'cwlitearm', None, None]
['30', 'run', 'avg_leaks', 'tvla', 'cwnano', 'ECB', '1000']
['30', 'run', 'n_traces', 'cpa', 'cwlitearm', 'ECB', None]
[None, None, 'n_traces', 'cpa', 'cwlitearm', None, None]
[None, None, 'avg_leaks', 'tvla', 'cwlitearm', 'CBC', None]
[None, None, 'avg_leaks', 'tvla', 'cwnano', 'CTR', None]


Import Results

In [62]:
sboxes_df.pop("box")
sboxes_df.pop("inverse")

sboxes_dict = sboxes_df.T.to_dict()
for i, filename in enumerate(files):
    col_name = col_names[i]
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            vals = re.split(r'\s', line)
            name = vals[0]
            result = vals[1]
            sboxes_dict[name][col_name] = result

sboxes_info_df = pd.DataFrame.from_dict(sboxes_dict).T

Round columns

In [63]:
sboxes_info_df = sboxes_info_df.apply(pd.to_numeric)
sboxes_info_df = sboxes_info_df.round(5)

Print to CSV

In [64]:
sboxes_info_df.to_csv("./sboxes_results.csv")